# Financial Statement Analysis

### Extract Balance Sheet

In [22]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as ur

In [29]:
index= 'FCL.NS'
url_is = 'https://finance.yahoo.com/quote/' + index + '/financials?p=' + index
url_bs = 'https://finance.yahoo.com/quote/' + index +'/balance-sheet?p=' + index
url_cf = 'https://finance.yahoo.com/quote/' + index + '/cash-flow?p='+ index

In [31]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

symbol = 'RELIANCE.NS'

url = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol

# Set up the request headers that we're going to use, to simulate
# a request by the Chrome browser. Simulating a request from a browser
# is generally good practice when building a scraper
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'close',
    'DNT': '1', # Do Not Track Request Header 
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

# Fetch the page that we're going to parse, using the request headers
# defined above
page = requests.get(url, headers=headers)

# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want
tree = html.fromstring(page.content)

# Smoke test that we fetched the page by fetching and displaying the H1 element
tree.xpath("//h1/text()")

['Reliance Industries Limited (RELIANCE.NS)']

In [32]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

# Ensure that some table rows are found; if none are found, then it's possible
# that Yahoo Finance has changed their page layout, or have detected
# that you're scraping the page.
assert len(table_rows) > 0

parsed_rows = []

for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    for rs in el:
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    if (none_count < 4):
        parsed_rows.append(parsed_row)

df = pd.DataFrame(parsed_rows)
df

,0,1,2,3,4
0,Breakdown,3/31/2023,3/31/2022,3/31/2021,3/31/2020
1,Total Assets,"17,135,060,000","14,996,650,000","13,212,120,000","11,659,150,000"
2,Total Liabilities Net Minority Interest,"7,793,440,000","6,106,810,000","5,217,800,000","7,045,680,000"
3,Total Equity Gross Minority Interest,"9,341,620,000","8,889,840,000","7,994,320,000","4,613,470,000"
4,Total Capitalization,"10,043,290,000","9,671,840,000","8,638,550,000","6,467,970,000"
5,Common Stock Equity,"8,211,530,000","7,794,850,000","7,001,720,000","4,491,660,000"
6,Capital Lease Obligations,"204,260,000","156,690,000","83,140,000","86,970,000"
7,Net Tangible Assets,"4,919,100,000","5,476,870,000","5,551,980,000","3,024,180,000"
8,Working Capital,"454,440,000","383,570,000","954,430,000","-1,546,560,000"
9,Invested Capital,"11,358,610,000","10,457,900,000","9,519,830,000","7,854,600,000"


In [34]:
df = pd.DataFrame(parsed_rows)
df = df.set_index(0)  # Set the index to the first column: 'Period Ending'.
df = df.transpose()  # Transpose the DataFrame, so that our header contains the account names

# Rename the "Breakdown" column to "Date"
cols = list(df.columns)
cols[0] = 'Date'
df.columns = cols  # Assign the modified column names directly

df

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number
1,3/31/2023,"17,135,060,000","7,793,440,000","9,341,620,000","10,043,290,000","8,211,530,000","204,260,000","4,919,100,000","454,440,000","11,358,610,000","4,919,100,000","3,351,340,000","2,400,000,000","6,766,094","6,766,094"
2,3/31/2022,"14,996,650,000","6,106,810,000","8,889,840,000","9,671,840,000","7,794,850,000","156,690,000","5,476,870,000","383,570,000","10,457,900,000","5,476,870,000","2,819,740,000","2,327,960,000","6,765,994","6,765,994"
3,3/31/2021,"13,212,120,000","5,217,800,000","7,994,320,000","8,638,550,000","7,001,720,000","83,140,000","5,551,980,000","954,430,000","9,519,830,000","5,551,980,000","2,601,250,000","2,373,050,000","6,762,069","6,762,069"
4,3/31/2020,"11,659,150,000","7,045,680,000","4,613,470,000","6,467,970,000","4,491,660,000","86,970,000","3,024,180,000","-1,546,560,000","7,854,600,000","3,024,180,000","3,449,910,000","3,104,910,000","6,399,357","6,399,357"


In [35]:

numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

for column_name in numeric_columns:
    df[column_name] = df[column_name].str.replace(',', '') # Remove the thousands separator
    df[column_name] = df[column_name].astype(np.float64) # Convert the column to float64

df.dtypes
     

Date                                        object
Total Assets                               float64
Total Liabilities Net Minority Interest    float64
Total Equity Gross Minority Interest       float64
Total Capitalization                       float64
Common Stock Equity                        float64
Capital Lease Obligations                  float64
Net Tangible Assets                        float64
Working Capital                            float64
Invested Capital                           float64
Tangible Book Value                        float64
Total Debt                                 float64
Net Debt                                   float64
Share Issued                               float64
Ordinary Shares Number                     float64
dtype: object

In [38]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'close',
        'DNT': '1', # Do Not Track Request Header 
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
    }

    return requests.get(url, headers=headers)

def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df.columns = cols  # Assign the modified column names directly

    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64
        
    return df

def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
        
    return df

In [39]:
df_income=scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)


In [40]:
df_income


,Date,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Pretax Income,Tax Provision,Net Income Common Stockholders,...,EBIT,EBITDA,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Total Unusual Items Excluding Goodwill,Total Unusual Items,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
1,ttm,8789980000.0,6185090000.0,2604890000.0,1504010000.0,1100880000.0,-231760000.0,1016430000.0,222210000.0,699690000.0,...,1248190000.0,1248190000.0,6185090000.0,NaN,702870000.0,0.0,0.0,1248190000.0,0.0,0.0
2,3/31/2023,8794680000.0,6295990000.0,2498690000.0,1472800000.0,1025890000.0,-195710000.0,948010000.0,207130000.0,667020000.0,...,1143720000.0,1546910000.0,6295990000.0,403190000.0,667020000.0,0.0,0.0,1546910000.0,0.0,0.0
3,3/31/2022,6999620000.0,5047090000.0,1952530000.0,1145900000.0,806630000.0,-145840000.0,841420000.0,162970000.0,607050000.0,...,987260000.0,1285230000.0,5047090000.0,297970000.0,607050000.0,28360000.0,28360000.0,1256870000.0,0.0,5492892.0
4,3/31/2021,4669240000.0,3390610000.0,1278630000.0,736980000.0,541650000.0,-76060000.0,554610000.0,17220000.0,491280000.0,...,733680000.0,999400000.0,3390610000.0,265720000.0,491280000.0,106060000.0,106060000.0,893340000.0,0.0,3659070.0
5,3/31/2020,5975350000.0,4514360000.0,1460990000.0,790890000.0,670100000.0,-103870000.0,536060000.0,137260000.0,393540000.0,...,734670000.0,956700000.0,4514360000.0,222030000.0,393540000.0,-25950000.0,-25950000.0,982650000.0,0.0,-6137175.0


In [41]:
df_income.columns


Index(['Date', 'Total Revenue', 'Cost of Revenue', 'Gross Profit',
       'Operating Expense', 'Operating Income',
       'Net Non Operating Interest Income Expense', 'Pretax Income',
       'Tax Provision', 'Net Income Common Stockholders',
       'Diluted NI Available to Com Stockholders', 'Basic Average Shares',
       'Diluted Average Shares', 'Rent Expense Supplemental', 'Total Expenses',
       'Net Income from Continuing & Discontinued Operation',
       'Normalized Income', 'Interest Income', 'Interest Expense',
       'Net Interest Income', 'EBIT', 'EBITDA', 'Reconciled Cost of Revenue',
       'Reconciled Depreciation',
       'Net Income from Continuing Operation Net Minority Interest',
       'Total Unusual Items Excluding Goodwill', 'Total Unusual Items',
       'Normalized EBITDA', 'Tax Rate for Calcs',
       'Tax Effect of Unusual Items'],
      dtype='object')

In [42]:

BS_analysis = df# copy columns of dataframe
BS_analysis

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number
1,3/31/2023,1.713506e+10,7.793440e+09,9.341620e+09,1.004329e+10,8.211530e+09,204260000.0,4.919100e+09,4.544400e+08,1.135861e+10,4.919100e+09,3.351340e+09,2.400000e+09,6766094.0,6766094.0
2,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.327960e+09,6765994.0,6765994.0
3,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0
4,3/31/2020,1.165915e+10,7.045680e+09,4.613470e+09,6.467970e+09,4.491660e+09,86970000.0,3.024180e+09,-1.546560e+09,7.854600e+09,3.024180e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0


## Working Capital Per Dollar of Sales = Working Capital ÷ Total Sales

#### Total Sales is from Income Statement

In [43]:
current_assets = BS_analysis['Total Assets']
current_liabilities = BS_analysis['Total Liabilities Net Minority Interest']
working_capital = current_assets - current_liabilities
working_capital
total_sales = df_income['Total Revenue']
working_capital_per_dollar_of_sales = working_capital / total_sales
BS_analysis['Working Capital per Dollar of Sales'] = working_capital # copy columns of dataframe
BS_analysis
     

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Working Capital per Dollar of Sales
1,3/31/2023,1.713506e+10,7.793440e+09,9.341620e+09,1.004329e+10,8.211530e+09,204260000.0,4.919100e+09,4.544400e+08,1.135861e+10,4.919100e+09,3.351340e+09,2.400000e+09,6766094.0,6766094.0,9.341620e+09
2,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.327960e+09,6765994.0,6765994.0,8.889840e+09
3,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0,7.994320e+09
4,3/31/2020,1.165915e+10,7.045680e+09,4.613470e+09,6.467970e+09,4.491660e+09,86970000.0,3.024180e+09,-1.546560e+09,7.854600e+09,3.024180e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0,4.613470e+09


## Current Ratio = Current Assets ÷ Current Liabilities

In [45]:

current_ratio = current_assets / current_liabilities
BS_analysis['Current Ratio'] = current_ratio
BS_analysis

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Working Capital per Dollar of Sales,Current Ratio
1,3/31/2023,1.713506e+10,7.793440e+09,9.341620e+09,1.004329e+10,8.211530e+09,204260000.0,4.919100e+09,4.544400e+08,1.135861e+10,4.919100e+09,3.351340e+09,2.400000e+09,6766094.0,6766094.0,9.341620e+09,2.198652
2,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.327960e+09,6765994.0,6765994.0,8.889840e+09,2.455726
3,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0,7.994320e+09,2.532125
4,3/31/2020,1.165915e+10,7.045680e+09,4.613470e+09,6.467970e+09,4.491660e+09,86970000.0,3.024180e+09,-1.546560e+09,7.854600e+09,3.024180e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0,4.613470e+09,1.654794


## debt-to-equity ratio = total liabilities ÷ shareholders' equity

In [46]:

total_liabilities = df['Total Liabilities Net Minority Interest']
shareholders_equity = df['Common Stock Equity']
debt2equity_ratio = total_liabilities / shareholders_equity
BS_analysis['Debt to Equity Ratio'] = debt2equity_ratio
BS_analysis

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Working Capital per Dollar of Sales,Current Ratio,Debt to Equity Ratio
1,3/31/2023,1.713506e+10,7.793440e+09,9.341620e+09,1.004329e+10,8.211530e+09,204260000.0,4.919100e+09,4.544400e+08,1.135861e+10,4.919100e+09,3.351340e+09,2.400000e+09,6766094.0,6766094.0,9.341620e+09,2.198652,0.949085
2,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.327960e+09,6765994.0,6765994.0,8.889840e+09,2.455726,0.783442
3,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0,7.994320e+09,2.532125,0.745217
4,3/31/2020,1.165915e+10,7.045680e+09,4.613470e+09,6.467970e+09,4.491660e+09,86970000.0,3.024180e+09,-1.546560e+09,7.854600e+09,3.024180e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0,4.613470e+09,1.654794,1.568614


## Receivable Turnover = Net Credit Sales ÷ Average Net Receivables for the Period

In [47]:
net_credit_sales = df_income['Net Income from Continuing Operation Net Minority Interest']
average_net_receivables_for_the_period = df['Tangible Book Value']
receivable_turnover = net_credit_sales / average_net_receivables_for_the_period
BS_analysis['Receivable Turnover'] = receivable_turnover
BS_analysis
     

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Working Capital per Dollar of Sales,Current Ratio,Debt to Equity Ratio,Receivable Turnover
1,3/31/2023,1.713506e+10,7.793440e+09,9.341620e+09,1.004329e+10,8.211530e+09,204260000.0,4.919100e+09,4.544400e+08,1.135861e+10,4.919100e+09,3.351340e+09,2.400000e+09,6766094.0,6766094.0,9.341620e+09,2.198652,0.949085,0.142886
2,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.327960e+09,6765994.0,6765994.0,8.889840e+09,2.455726,0.783442,0.121789
3,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0,7.994320e+09,2.532125,0.745217,0.109339
4,3/31/2020,1.165915e+10,7.045680e+09,4.613470e+09,6.467970e+09,4.491660e+09,86970000.0,3.024180e+09,-1.546560e+09,7.854600e+09,3.024180e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0,4.613470e+09,1.654794,1.568614,0.162451
